In [ ]:
# ===================================================================
# SEL 1: KONFIGURASI EKSPERIMEN (Pusat Kendali Anda)
# ===================================================================
import os

# --- Atur variabel lingkungan (HARUS PALING ATAS) ---
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# --- 1. PILIH PENGATURAN EKSPERIMEN ANDA DI SINI ---
MODEL_BASE_NAME = "ssdmobilenetv3_statis_baseline_albuminatinon_test"
EPOCHS = 70
BATCH_SIZE = 8
LEARNING_RATE = 0.001
# Catatan: Jika BS=8, LR=0.002. Jika BS=16, LR=0.004

# --- 2. PENGATURAN PROYEK (biasanya tidak perlu diubah) ---
DATASET_ROOT = 'dataset'
CLASSES_TO_USE = [
    'baguette', 'cornbread', 'croissant', 'ensaymada', 'flatbread',
    'sourdough', 'wheat-bread', 'white-bread', 'whole-grain-bread', 'pandesal'
]
NUM_CLASSES = len(CLASSES_TO_USE)

# --- 3. NAMA FOLDER DAN PATH DIBUAT SECARA OTOMATIS ---
lr_str = str(LEARNING_RATE).replace('.', '_')
PROJECT_NAME = f"{MODEL_BASE_NAME}_e{EPOCHS}_bs{BATCH_SIZE}_lr{lr_str}"
WORK_DIR = f"outputs/{PROJECT_NAME}"

# Cetak konfigurasi untuk verifikasi
print("="*60)
print("KONFIGURASI EKSPERIMEN AKTIF:")
print(f"  - Nama Proyek: {PROJECT_NAME}")
print(f"  - Direktori Kerja: {WORK_DIR}")
print(f"  - Epoch: {EPOCHS}")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Learning Rate: {LEARNING_RATE}")
print("="*60)

In [ ]:
# ===================================================================
# SEL 2: INSTALASI, IMPORT, DAN PERSIAPAN HELPER (VERSI BASELINE)
# ===================================================================
import sys, json, torch, numpy as np, pandas as pd, requests
from PIL import Image
import matplotlib
matplotlib.use('Agg')

# --- 1. MEMBUAT FILE transforms.py KUSTOM (HANYA FLIP & TENSOR) ---
content_transforms = r"""
import random
import torchvision.transforms.functional as F
import torch
from torchvision.transforms import ToTensor as _ToTensor

class Compose:
    def __init__(self, transforms): self.transforms = transforms
    def __call__(self, image, target):
        for t in self.transforms: image, target = t(image, target)
        return image, target

class ToTensor(_ToTensor):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

class RandomHorizontalFlip:
    def __init__(self, prob=0.5): self.prob = prob
    def __call__(self, image, target):
        if random.random() < self.prob:
            width, _ = image.size
            image = F.hflip(image)
            if "boxes" in target:
                bbox = target["boxes"]
                bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
                target["boxes"] = bbox
        return image, target
"""
try:
    with open('transforms.py', 'w') as f: f.write(content_transforms.strip())
    print("File 'transforms.py' kustom (baseline) berhasil dibuat.")
except Exception as e:
    print(f"GAGAL membuat 'transforms.py': {e}")
    
# --- Sisa dari SEL 2 (tidak perlu diubah, bisa disalin dari versi final sebelumnya) ---
# ... (Download utils.py, coco_utils.py, engine_robust.py, install pycocotools, dst.)
# Untuk kejelasan, berikut kode lengkapnya:

helper_files_to_download = {
    'utils.py': 'https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py',
    'engine_robust.py': 'https://gist.githubusercontent.com/renton-k5/7334bfe4352b2f6f1a8f815041071d72/raw/4373ca35d6c81308311a2f00d38a0b3b4f9f0612/engine_robust.py',
    'coco_utils.py': 'https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py',
    'coco_eval.py': 'https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py'
}
print("\nMendownload sisa file helper...")
sys.path.insert(0, os.path.abspath('.'))
for filename, url in helper_files_to_download.items():
    if not os.path.exists(filename): # Hanya download jika belum ada
        try:
            r = requests.get(url)
            r.raise_for_status()
            with open(filename, 'w', encoding='utf-8') as f: f.write(r.text)
            print(f"File {filename} berhasil di-download.")
        except Exception as e:
            print(f"GAGAL men-download {filename}: {e}")
    else:
        print(f"File {filename} sudah ada, download dilewati.")
print("\nMenginstall pycocotools...")
!pip install pycocotools -q
print("pycocotools terinstall.")
import torchvision
from torchvision.datasets import CocoDetection
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
from torchvision.models.detection.ssdlite import SSDLiteClassificationHead
from engine_robust import train_one_epoch, evaluate_robust as evaluate
print(f"\nPyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# ===================================================================
# SEL 3: PEMROSESAN DATA (FILTER ANOTASI)
# ===================================================================
def filter_coco_annotations(original_ann_file, new_ann_file, classes_to_keep):
    if os.path.exists(new_ann_file):
        print(f"File anotasi terfilter sudah ada: {new_ann_file}")
        return
    with open(original_ann_file, 'r') as f: coco_data = json.load(f)
    print(f"Memfilter {original_ann_file}...")
    original_categories = {cat['name']: cat['id'] for cat in coco_data['categories']}
    new_categories, old_id_to_new_id, new_cat_id = [], {}, 1
    for cat_name in classes_to_keep:
        if cat_name in original_categories:
            new_categories.append({'id': new_cat_id, 'name': cat_name, 'supercategory': 'object'})
            old_id_to_new_id[original_categories[cat_name]] = new_cat_id
            new_cat_id += 1
    new_annotations, image_ids_with_annotations = [], set()
    for ann in coco_data['annotations']:
        if ann.get('iscrowd', 0) == 0 and ann['category_id'] in old_id_to_new_id:
            ann['category_id'] = old_id_to_new_id[ann['category_id']]
            new_annotations.append(ann)
            image_ids_with_annotations.add(ann['image_id'])
    new_images = [img for img in coco_data['images'] if img['id'] in image_ids_with_annotations]
    new_coco_data = {'images': new_images, 'annotations': new_annotations, 'categories': new_categories}
    os.makedirs(os.path.dirname(new_ann_file), exist_ok=True)
    with open(new_ann_file, 'w') as f: json.dump(new_coco_data, f, indent=2)
    print(f"File anotasi baru disimpan di: {new_ann_file}")

original_train_ann = os.path.join(DATASET_ROOT, 'train', '_annotations.coco.json')
filtered_train_ann = os.path.join(DATASET_ROOT, 'train', 'filtered_annotations.coco.json')
original_valid_ann = os.path.join(DATASET_ROOT, 'valid', '_annotations.coco.json')
filtered_valid_ann = os.path.join(DATASET_ROOT, 'valid', 'filtered_annotations.coco.json')

filter_coco_annotations(original_train_ann, filtered_train_ann, CLASSES_TO_USE)
filter_coco_annotations(original_valid_ann, filtered_valid_ann, CLASSES_TO_USE)

In [ ]:
# ===================================================================
# SEL 4: PEMBUATAN SKRIP train.py (DIPERBAIKI DENGAN ALBUMENTATIONS)
# ===================================================================

script_content = r"""
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' 
import sys, json, torch, numpy as np, pandas as pd, gc, argparse, cv2
import torchvision
from torchvision.datasets import CocoDetection
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
from torchvision.models.detection.ssdlite import SSDLiteClassificationHead
import torch.utils.data
sys.path.insert(0, os.path.abspath('.'))
import utils
from engine_robust import train_one_epoch, evaluate_robust as evaluate

# --- PERUBAHAN DIMULAI DI SINI: Integrasi Albumentations ---
import albumentations as A
from albumentations.pytorch import ToTensorV2
def get_transform(train):
    if train:
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            # Tambahkan rotasi. Sangat penting untuk mengatasi FN.
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.6),
            
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
            A.GaussianBlur(blur_limit=(3, 7), p=0.4),
            
            # CoarseDropout memaksa model melihat bagian roti yang berbeda
            A.CoarseDropout(max_holes=8, max_height=25, max_width=25, p=0.5),

            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(
            format='coco',
            label_fields=['category_ids'],
            min_visibility=0.2
        ))
    else:
        # Validasi tetap sama
        return A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

class BreadDataset(CocoDetection):
    def __init__(self, root, annFile, transform=None):
        super(BreadDataset, self).__init__(root, annFile)
        self.transform = transform

    def __getitem__(self, idx):
        img, target_list = super(BreadDataset, self).__getitem__(idx)
        
        # Konversi PIL Image ke NumPy array (wajib untuk Albumentations)
        image = np.array(img)
        
        boxes, labels, areas = [], [], []
        
        for ann in target_list:
            x, y, w, h = ann['bbox']
            # Filter anotasi yang tidak valid dari awal
            if w > 0 and h > 0:
                boxes.append([x, y, w, h])
                labels.append(ann['category_id'])
                areas.append(ann['area'])
        
        # Siapkan dictionary untuk augmentasi
        transform_input = {
            'image': image,
            'bboxes': boxes,
            'category_ids': labels
        }
        
        if self.transform:
            transformed = self.transform(**transform_input)
            image = transformed['image']
            transformed_boxes = transformed['bboxes']
            transformed_labels = transformed['category_ids']
        else:
            transformed_boxes = boxes
            transformed_labels = labels

        target = {}
        # Menangani kasus di mana semua bounding box hilang setelah augmentasi
        if len(transformed_boxes) > 0:
            # Konversi format [x,y,w,h] ke [x1,y1,x2,y2] yang dibutuhkan model
            boxes_xyxy = [[box[0], box[1], box[0] + box[2], box[1] + box[3]] for box in transformed_boxes]
            target["boxes"] = torch.as_tensor(boxes_xyxy, dtype=torch.float32)
            target["labels"] = torch.as_tensor(transformed_labels, dtype=torch.int64)
            # Buat ulang area jika perlu (opsional, tergantung evaluator)
            target["area"] = torch.as_tensor([(b[2]-b[0])*(b[3]-b[1]) for b in boxes_xyxy], dtype=torch.float32)
        else: 
            # Jika tidak ada box, buat tensor kosong untuk mencegah error
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros(0, dtype=torch.int64)
            target["area"] = torch.zeros(0, dtype=torch.float32)

        target["image_id"] = torch.tensor([self.ids[idx]])
        target["iscrowd"] = torch.zeros((len(transformed_boxes),), dtype=torch.int64)

        return image, target
# --- PERUBAHAN SELESAI ---

def freeze_bn(module):
    if isinstance(module, torch.nn.BatchNorm2d):
        module.eval()
        if module.weight is not None: module.weight.requires_grad = False
        if module.bias is not None: module.bias.requires_grad = False

def main(args):
    model_base_name = args.model_base_name
    lr_str = str(args.lr).replace('.', '_')
    PROJECT_NAME = f"{model_base_name}_e{args.epochs}_bs{args.batch_size}_lr{lr_str}"
    DATASET_ROOT = 'dataset'
    CLASSES_TO_USE = ['baguette', 'cornbread', 'croissant', 'ensaymada', 'flatbread', 'sourdough', 'wheat-bread', 'white-bread', 'whole-grain-bread', 'pandesal']
    NUM_CLASSES = len(CLASSES_TO_USE)
    WORK_DIR = f"outputs/{PROJECT_NAME}"
    os.makedirs(WORK_DIR, exist_ok=True)
    print("="*60 + f"\nMemulai Eksperimen: {PROJECT_NAME}\n" + "="*60)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    dataset = BreadDataset(os.path.join(DATASET_ROOT, 'train'), os.path.join(DATASET_ROOT, 'train', 'filtered_annotations.coco.json'), transform=get_transform(train=True))
    dataset_test = BreadDataset(os.path.join(DATASET_ROOT, 'valid'), os.path.join(DATASET_ROOT, 'valid', 'filtered_annotations.coco.json'), transform=get_transform(train=False))
    
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size, shuffle=True, num_workers=2, collate_fn=utils.collate_fn, drop_last=True)
    data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=2, collate_fn=utils.collate_fn)
    
    model = ssdlite320_mobilenet_v3_large(weights='DEFAULT')
    num_anchors = model.anchor_generator.num_anchors_per_location()
    in_channels = [m[0][0].in_channels for m in model.head.classification_head.module_list]
    new_head = SSDLiteClassificationHead(in_channels=in_channels, num_anchors=num_anchors, num_classes=(NUM_CLASSES + 1), norm_layer=torch.nn.BatchNorm2d)
    model.head.classification_head = new_head
    model.to(device)
    
    # SARAN: Coba jalankan SATU KALI TANPA freeze_bn untuk melihat perbedaannya
    # model.apply(freeze_bn) 
    
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=args.lr, momentum=0.9, weight_decay=0.0005)
    
    # --- AKTIFKAN KEMBALI LR SCHEDULER ---
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs, eta_min=1e-6)
    
    print(f"--- MEMULAI TRAINING {args.epochs} EPOCH (DENGAN ALBUMENTATIONS & LR SCHEDULER) ---")
    best_map = 0.0
    training_history = []
    for epoch in range(args.epochs):
        metric_logger = train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=50)
        
        # Panggil scheduler setelah setiap epoch training
        lr_scheduler.step()
        
        eval_result = evaluate(model, data_loader_test, device=device)
        current_map = eval_result.coco_eval['bbox'].stats[0]
        # Catat learning rate yang sekarang digunakan
        current_lr = optimizer.param_groups[0]["lr"]
        training_history.append({'epoch': epoch + 1, 'loss': metric_logger.meters['loss'].global_avg, 'mAP_0.50:0.95': current_map, 'lr': current_lr})
        print(f"Epoch {epoch+1}/{args.epochs}: Avg Loss={metric_logger.meters['loss'].global_avg:.4f}, mAP={current_map:.4f}, LR={current_lr:.6f}")
        
        if current_map > best_map:
            best_map = current_map
            utils.save_on_master(model.state_dict(), os.path.join(WORK_DIR, 'best_model.pth'))
            print(f"*** Best mAP updated: {best_map:.4f} (model saved) ***")
    
    print(f"\n--- TRAINING SELESAI --- Best mAP: {best_map:.4f}")
    pd.DataFrame(training_history).to_csv(os.path.join(WORK_DIR, 'training_log.csv'), index=False)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model-base-name', type=str, default="ssdmobilenetv3")
    parser.add_argument('--epochs', type=int, default=70)
    parser.add_argument('--batch-size', type=int, default=8)
    parser.add_argument('--lr', type=float, default=0.001)
    args = parser.parse_args()
    main(args)
"""

try:
    with open('train.py', 'w', encoding='utf-8') as f: f.write(script_content.strip())
    print(">>> Skrip 'train.py' (diperbaiki dengan Albumentations & LR Scheduler) berhasil dibuat.")
except Exception as e:
    print(f">>> GAGAL membuat skrip 'train.py': {e}")

In [ ]:
# ===================================================================
# SEL 5: JALANKAN TRAINING (VERSI DIPERBAIKI)
# ===================================================================
import subprocess
import sys  # <-- TAMBAHKAN BARIS INI

# Perintah ini akan secara otomatis menggunakan variabel dari SEL 1
command = [
    sys.executable, "-u", "train.py",
    "--model-base-name", MODEL_BASE_NAME,
    "--epochs", str(EPOCHS),
    "--batch-size", str(BATCH_SIZE),
    "--lr", str(LEARNING_RATE)
]

# Jalankan proses
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', bufsize=1)
print(f"--- Memulai eksekusi: {' '.join(command)} ---")
if process.stdout:
    for line in iter(process.stdout.readline, ''): print(line, end='')
return_code = process.wait()
print(f"\n--- Eksekusi selesai dengan kode: {return_code} ---")

In [ ]:
# ===================================================================
# SEL 6: ANALISIS HASIL LENGKAP (PLOT, INFERENSI, METRIK) - FIX NameError
# ===================================================================
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
from sklearn.metrics import precision_recall_curve
from tqdm import tqdm
import torchvision
from torchvision.datasets import CocoDetection
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
from torchvision.models.detection.ssdlite import SSDLiteClassificationHead

import transforms as CustomT
import utils # <-- PERBAIKAN: TAMBAHKAN IMPORT INI

# --- 1. PLOTTING KURVA STABILITAS ---
# ... (sisa kode plotting tidak berubah) ...
log_path = os.path.join(WORK_DIR, 'training_log.csv')
print(f"\n>>> Membuat grafik dari: {log_path}...")
if os.path.exists(log_path):
    log_df = pd.read_csv(log_path)
    fig, ax1 = plt.subplots(figsize=(14, 7))
    ax1.set_xlabel('Epoch'); ax1.set_ylabel('Rata-rata Training Loss', color='tab:red')
    ax1.plot(log_df['epoch'], log_df['loss'], 'o-', color='tab:red')
    ax1.tick_params(axis='y', labelcolor='tab:red'); ax1.grid(True, linestyle='--', alpha=0.6)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Validation mAP (0.50:0.95)', color='tab:blue')
    ax2.plot(log_df['epoch'], log_df['mAP_0.50:0.95'], 's-', color='tab:blue')
    ax2.tick_params(axis='y', labelcolor='tab:blue')
    plt.title(f'Kurva Stabilitas Training: {PROJECT_NAME}')
    plot_path = os.path.join(WORK_DIR, 'training_stability_curve.png')
    plt.savefig(plot_path, dpi=200)
    print(f">>> Grafik disimpan di: {plot_path}")
    plt.close()
else:
    print("File log tidak ditemukan. Plotting dilewati.")


# --- 2. PERSIAPAN UNTUK ANALISIS LANJUTAN ---
best_model_path = os.path.join(WORK_DIR, 'best_model.pth')
REPORT_DIR = os.path.join(WORK_DIR, 'post_training_reports')
os.makedirs(REPORT_DIR, exist_ok=True)

if os.path.exists(best_model_path):
    # --- A. BUAT ULANG SEMUA KOMPONEN YANG DIBUTUHKAN ---
    print(f"\nModel terbaik ditemukan. Mempersiapkan untuk analisis mendalam...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Buat ulang model dengan arsitektur yang benar, lalu muat bobot
    model = ssdlite320_mobilenet_v3_large(weights='DEFAULT')
    num_anchors = model.anchor_generator.num_anchors_per_location()
    in_channels = [m[0][0].in_channels for m in model.head.classification_head.module_list]
    new_head = SSDLiteClassificationHead(in_channels=in_channels, num_anchors=num_anchors, num_classes=(NUM_CLASSES + 1), norm_layer=torch.nn.BatchNorm2d)
    model.head.classification_head = new_head
    model.load_state_dict(torch.load(best_model_path))
    model.to(device)
    model.eval()
    print(f"Model berhasil dimuat dari: {best_model_path}")
    
    # --- B. INFERENSI PADA GAMBAR ACAK (DENGAN NAMA FILE AMAN) ---
    print("\n>>> Melakukan inferensi pada gambar acak...")
    INFERENCE_DIR = os.path.join(WORK_DIR, 'inference_results'); os.makedirs(INFERENCE_DIR, exist_ok=True)
    valid_img_dir = os.path.join(DATASET_ROOT, 'valid')
    all_valid_imgs = [f for f in os.listdir(valid_img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if all_valid_imgs:
        random_samples = random.sample(all_valid_imgs, k=min(5, len(all_valid_imgs)))
        category_map = {i+1: name for i, name in enumerate(CLASSES_TO_USE)}
        
        with torch.no_grad():
            for img_name in random_samples:
                img_path = os.path.join(valid_img_dir, img_name)
                img_pil = Image.open(img_path).convert("RGB")
                
                img_tensor, _ = CustomT.Compose([CustomT.ToTensor()])(img_pil, None)
                prediction = model(img_tensor.unsqueeze(0).to(device))[0]
                
                original_image_cv = cv2.imread(img_path)
                score_thr = 0.5
                
                detections = []
                for i in range(len(prediction['scores'])):
                    score = prediction['scores'][i].item()
                    if score > score_thr:
                        box = [int(coord) for coord in prediction['boxes'][i].tolist()]
                        label_id = prediction['labels'][i].item()
                        class_name = category_map.get(label_id, 'N/A')
                        detections.append({'name': class_name, 'score': score})
                        
                        label_text = f"{class_name}: {score:.2f}"
                        cv2.rectangle(original_image_cv, (box[0], box[1]), (box[2], box[3]), (36, 255, 12), 2)
                        cv2.putText(original_image_cv, label_text, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)
                
                vis_path = os.path.join(INFERENCE_DIR, f"visualization_{img_name}")
                cv2.imwrite(vis_path, original_image_cv)
                
                base_name = os.path.splitext(img_name)[0]
                sanitized_name = base_name.replace('.', '_')
                summary_path = os.path.join(INFERENCE_DIR, f"summary_{sanitized_name}.txt")

                with open(summary_path, 'w') as f:
                    f.write(f"Hasil Deteksi pada: {img_name}\n")
                    f.write(f"Score Threshold: {score_thr}\n" + "="*30 + "\n")
                    if not detections:
                        f.write("Tidak ada objek terdeteksi.\n")
                    else:
                        for det in sorted(detections, key=lambda x: x['score'], reverse=True):
                            f.write(f"- {det['name']}: {det['score']:.2f}\n")
                    f.write("="*30 + f"\nTOTAL TERDETEKSI: {len(detections)}\n")

        print(f">>> Hasil inferensi (gambar & teks) disimpan di: {INFERENCE_DIR}")

    # --- C. ANALISIS MENDALAM (METRIK PER KELAS, DLL.) ---
    print("\n>>> Memulai evaluasi kustom untuk metrik per kelas...")
    
    class AnalysisBreadDataset(CocoDetection):
        def __init__(self, root, annFile, transform=None):
            super().__init__(root, annFile); self.transform = transform
        def __getitem__(self, idx):
            img, target_list = super().__getitem__(idx)
            boxes, labels = [], []
            for ann in target_list:
                x, y, w, h = ann['bbox']
                if w>0 and h>0: boxes.append([x, y, x+w, y+h]); labels.append(ann['category_id'])
            target = {"boxes": torch.as_tensor(boxes, dtype=torch.float32), "labels": torch.as_tensor(labels, dtype=torch.int64)}
            if self.transform: img, target = self.transform(img, target)
            target["image_id"] = torch.tensor([self.ids[idx]])
            return img, target

    def get_analysis_transform(): return CustomT.Compose([CustomT.ToTensor()])
            
    dataset_test = AnalysisBreadDataset(os.path.join(DATASET_ROOT, 'valid'), os.path.join(DATASET_ROOT, 'valid', 'filtered_annotations.coco.json'), transform=get_analysis_transform())
    
    # Sekarang 'utils' sudah terdefinisi
    data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=0, collate_fn=utils.collate_fn)
    
    # ... (sisa kode analisis mendalam tidak berubah) ...
    n_classes = len(CLASSES_TO_USE)
    conf_mat = np.zeros((n_classes, n_classes), dtype=np.int32)
    per_class_counts = {'TP': np.zeros(n_classes), 'FP': np.zeros(n_classes), 'FN': np.zeros(n_classes)}
    pr_store = {i: {'scores': [], 'match': []} for i in range(n_classes)}
    iou_thr, score_thr = 0.5, 0.5
    with torch.no_grad():
        for images, targets in tqdm(data_loader_test, desc="Mengevaluasi gambar"):
            outputs = model([img.to(device) for img in images])
            for target, output in zip(targets, outputs):
                gt_boxes, gt_labels = target['boxes'].numpy(), target['labels'].numpy()
                keep = output['scores'] > score_thr
                pred_boxes, pred_labels, pred_scores = output['boxes'][keep].cpu().numpy(), output['labels'][keep].cpu().numpy(), output['scores'][keep].cpu().numpy()
                gt_matched = [False] * len(gt_boxes)
                for i in range(len(pred_boxes)):
                    best_iou, best_gt_idx = 0, -1
                    for j in range(len(gt_boxes)):
                        if gt_labels[j] == pred_labels[i]:
                            iou = torchvision.ops.box_iou(torch.from_numpy(pred_boxes[i:i+1]), torch.from_numpy(gt_boxes[j:j+1])).item()
                            if iou > best_iou: best_iou, best_gt_idx = iou, j
                    pred_class_idx = pred_labels[i] - 1
                    if best_iou >= iou_thr and best_gt_idx != -1 and not gt_matched[best_gt_idx]:
                        gt_class_idx = gt_labels[best_gt_idx] - 1
                        if 0 <= pred_class_idx < n_classes and 0 <= gt_class_idx < n_classes:
                            conf_mat[gt_class_idx, pred_class_idx] += 1
                            per_class_counts['TP'][pred_class_idx] += 1
                            pr_store[pred_class_idx]['match'].append(1)
                        gt_matched[best_gt_idx] = True
                    else:
                        if 0 <= pred_class_idx < n_classes:
                            per_class_counts['FP'][pred_class_idx] += 1
                            pr_store[pred_class_idx]['match'].append(0)
                    if 0 <= pred_class_idx < n_classes:
                        pr_store[pred_class_idx]['scores'].append(pred_scores[i])
                for j in range(len(gt_boxes)):
                    if not gt_matched[j]:
                        gt_class_idx = gt_labels[j] - 1
                        if 0 <= gt_class_idx < n_classes:
                            per_class_counts['FN'][gt_class_idx] += 1
    eps = 1e-12
    prec = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FP'] + eps)
    rec = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FN'] + eps)
    f1 = 2 * prec * rec / (prec + rec + eps)
    metrics_df = pd.DataFrame({'Kelas': CLASSES_TO_USE, 'TP': per_class_counts['TP'], 'FP': per_class_counts['FP'], 'FN': per_class_counts['FN'], 'Precision': prec, 'Recall': rec, 'F1-Score': f1})
    metrics_df.to_csv(os.path.join(REPORT_DIR, "per_class_metrics.csv"), index=False)
    print(f"\n>>> Metrik per kelas disimpan di: {REPORT_DIR}\n{metrics_df}")
    plt.figure(figsize=(12, 10)); sns.heatmap(conf_mat, annot=True, fmt='d', cmap='viridis', xticklabels=CLASSES_TO_USE, yticklabels=CLASSES_TO_USE)
    plt.title(f"Confusion Matrix (IoU>{iou_thr}, Score>{score_thr})"); plt.ylabel('True Label'); plt.xlabel('Predicted Label'); plt.tight_layout()
    cm_path = os.path.join(REPORT_DIR, "confusion_matrix.png"); plt.savefig(cm_path, dpi=200); plt.close()
    print(f"\n>>> Confusion matrix disimpan di: {cm_path}")
    pr_curve_dir = os.path.join(REPORT_DIR, 'pr_curves'); os.makedirs(pr_curve_dir, exist_ok=True)
    for i, class_name in enumerate(CLASSES_TO_USE):
        scores = np.array(pr_store[i]['scores']); matches = np.array(pr_store[i]['match'])
        if len(scores) == 0: continue
        precision_vals, recall_vals, _ = precision_recall_curve(matches, scores)
        plt.figure(); plt.plot(recall_vals, precision_vals, '-'); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR Curve - {class_name}"); plt.grid(); plt.xlim(-0.05, 1.05); plt.ylim(-0.05, 1.05)
        pr_path = os.path.join(pr_curve_dir, f"pr_curve_{i:02d}_{class_name.replace(' ','_')}.png"); plt.savefig(pr_path, dpi=200); plt.close()
    print(f">>> PR curves disimpan di: {pr_curve_dir}")
else:
    print(f"Analisis dilewati karena checkpoint terbaik tidak ditemukan di: {best_model_path}")